In [49]:
from classe_maze import *

# SAE A Maze In Python

# **1 Préambule**

SAE à rendre pour au plus tard le <u>**15 mars 2023 a 23h59**</u>.

# **2 Modélisation d’un labyrinthe**

<u> Sommets du graphe : </u>

(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)

<u>Arrêtes du graphe :</u>

{(0, 0), (1, 0)}, {(1, 0), (2, 0)}, {(2, 0), (3, 0)}, {(2, 0), (2, 1)}, ...

<u>Successeurs du graphes :</u>

{ (0, 0): {(1, 0)}, (0, 1): {(0, 2), (1, 1)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(2, 3), (0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)} }


# **3 Implémentation**

#### **La classe Maze par défaut**

<u>**Attributs de la classe :**</u>

**height**, le nombre de lignes (int) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules).

**width**, le nombre de colonnes (int) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules).

**neighbors** : un dictionnaire (dict) qui associe à chaque cellule, un set contenant ses voisines (c’est-à-dire les cellules qu’on peut atteindre en un déplacement1, sans être bloqué par un mur).

<u>**Rappel du code :**</u>

    class Maze:
        """
        Classe Labyrinthe
        Représentation sous forme de graphe non-orienté
        dont chaque sommet est une cellule (un tuple (l,c))
        et dont la structure est représentée par un dictionnaire
        - clés : sommets
        - valeurs : ensemble des sommets voisins accessibles
        """
        def __init__(self, height, width):
            """
            Constructeur d'un labyrinthe de height cellules de haut 
            et de width cellules de large 
            Les voisinages sont initialisés à des ensembles vides
            Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
            """
            self.height    = height
            self.width     = width
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

        def info(self):
            """
            **NE PAS MODIFIER CETTE MÉTHODE**
            Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
            Retour:
                chaîne (string): description textuelle des attributs de l'objet
            """
            txt = "**Informations sur le labyrinthe**\n"
            txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
            txt += "- Voisinages :\n"
            txt += str(self.neighbors)+"\n"
            valid = True
            for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
                for c2 in self.neighbors[c1]:
                    if c1 not in self.neighbors[c2]:
                        valid = False
                        break
                else:
                    continue
                break
            txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
            return txt

        def __str__(self):
            """
            **NE PAS MODIFIER CETTE MÉTHODE**
            Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
            Retour:
                chaîne (str) : chaîne de caractères représentant le labyrinthe
            """
            txt = ""
            # Première ligne
            txt += "┏"
            for j in range(self.width-1):
                txt += "━━━┳"
            txt += "━━━┓\n"
            txt += "┃"
            for j in range(self.width-1):
                txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
            txt += "   ┃\n"
            # Lignes normales
            for i in range(self.height-1):
                txt += "┣"
                for j in range(self.width-1):
                    txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
                txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
                txt += "┃"
                for j in range(self.width):
                    txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
                txt += "\n"
            # Bas du tableau
            txt += "┗"
            for i in range(self.width-1):
                txt += "━━━┻"
            txt += "━━━┛\n"

            return txt

<br>
<br>

#### **Ajouter / Enlever un mur**


- **Ajouter** un mur entre une cellule c1 et une cellule c2 revient à **diminuer deux** voisinages, d’une cellule chacun ; il faut retirer c1 du voisinage de c2 **et** retirer c2 du voisinage de c1 .  

> ``laby.neighbors[(i, j)].remove((y,x))``

- **Casser** un mur entre une cellule c1 et une cellule c2 revient à **augmenter deux** voisinages ; il faut ajouter c1 au voisinage de c2 **et** ajouter c2 au voisinage de c1.  

> ``laby.neighbors[(i, j)].add((y,x))``

La méthode **info()** fournie teste la cohérence des voisinages en vérifiant que, dès lors qu’une cellule c1 est dans le voisinage d’une cellule c2, alors c2 est aussi dans le voisinage de c1.

<br>
<br>

#### **A Faire**

Modifier le constructeur par défaut en lui ajoutant l’argument **empty**, un booléen qui indique si le graphe doit être créé avec aucun mur, ou avec tous les murs.  

Modifier le corps de la méthode de telle manière que :  

- si **empty** vaut **True**, chaque cellule a pour voisines celles qui lui sont contigües dans la grille ;
- si **empty** vaut **False**, aucune cellule n’a de voisines.  

<br>

In [ ]:
laby = Maze(5, 5, empty = True)
print(laby.neighbors)
print(laby)

{(0, 0): {(0, 1), (1, 0)}, (0, 1): {(1, 1), (0, 2), (0, 0)}, (0, 2): {(0, 1), (1, 2), (0, 3)}, (0, 3): {(1, 3), (0, 2), (0, 4)}, (0, 4): {(0, 3), (1, 4)}, (1, 0): {(1, 1), (2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 0), (1, 2), (2, 1)}, (1, 2): {(1, 1), (0, 2), (1, 3), (2, 2)}, (1, 3): {(2, 3), (1, 2), (0, 3), (1, 4)}, (1, 4): {(1, 3), (2, 4), (0, 4)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(3, 1), (1, 1), (2, 0), (2, 2)}, (2, 2): {(2, 3), (3, 2), (1, 2), (2, 1)}, (2, 3): {(2, 4), (3, 3), (1, 3), (2, 2)}, (2, 4): {(2, 3), (3, 4), (1, 4)}, (3, 0): {(3, 1), (4, 0), (2, 0)}, (3, 1): {(3, 2), (4, 1), (2, 1), (3, 0)}, (3, 2): {(3, 1), (3, 3), (4, 2), (2, 2)}, (3, 3): {(2, 3), (3, 2), (3, 4), (4, 3)}, (3, 4): {(4, 4), (2, 4), (3, 3)}, (4, 0): {(4, 1), (3, 0)}, (4, 1): {(3, 1), (4, 0), (4, 2)}, (4, 2): {(3, 2), (4, 1), (4, 3)}, (4, 3): {(4, 4), (3, 3), (4, 2)}, (4, 4): {(3, 4), (4, 3)}}
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫

In [ ]:
laby = Maze(5, 5, empty = False)
print(laby.neighbors)
print(laby)

{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (0, 4): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (1, 4): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (2, 4): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set(), (3, 4): set(), (4, 0): set(), (4, 1): set(), (4, 2): set(), (4, 3): set(), (4, 4): set()}
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛

